# Integrated Simulations

In [ ]:
import os; from os import listdir; from os.path import isfile, join
import re  
from skimage.io import imread
from scipy import ndimage
import trackpy as tp
from cellpose import models
from cellpose import plot
from matplotlib import gridspec
import math
import seaborn as sn                       
from joblib import Parallel, delayed
import multiprocessing
from numpy import unravel_index
# To manipulate arrays
import numpy as np 
import random
from tqdm.notebook import tqdm
from timeit import default_timer as timer
import scipy
import pandas as pd
import shutil
import pathlib
import tifffile
import sys
from skimage.exposure import rescale_intensity
import rsnapsim as rss
# Importing libraries with the watershed algorithm and local maximum detection
from scipy import ndimage as ndi              # Distance Transform
from skimage.feature import peak_local_max    # Local maxima in a matrix
from skimage.segmentation import watershed    # Watershed algorithm
from skimage.filters import gaussian # Module working with a gaussian filter
from skimage.draw import polygon
from skimage import measure
# Plotting
import matplotlib as mpl ; import matplotlib.pyplot as plt 
# To create interactive elements
import ipywidgets as widgets ; from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox, Layout, GridspecLayout ; from IPython.display import Image

In [ ]:
# Defining directories
current_dir = pathlib.Path().absolute()
sequences_dir = current_dir.parents[1].joinpath('DataBases','gene_files')
video_dir = current_dir.parents[1].joinpath('DataBases','moving_cells')
masks_dir = current_dir.parents[1].joinpath('DataBases','masks_for_sim_cell')
trajectories_dir = current_dir.parents[1].joinpath('DataBases','rsnapsim_simulations','kdm5b_ssa.npy')
rsnaped_dir = current_dir.parents[1].joinpath('rsnaped')
gene_file = current_dir.parents[1].joinpath('DataBases','gene_files','KDM5B_withTags.txt')

In [ ]:
#import rsnaped as rsp
sys.path.append(str(rsnaped_dir))
import rsnaped as rsp
rsp.Banner(show=True).print_banner()

In [ ]:
# List all video in dir
list_files_names = sorted([f for f in listdir(video_dir) if isfile(join(video_dir, f)) and ('.tif') in f], key=str.lower)  # reading all tif files in the folder
list_files_names.sort(key=lambda f: int(re.sub('\D', '', f)))  # sorting the index in numerical order
path_files = [ str(video_dir.joinpath(f).resolve()) for f in list_files_names ] # creating the complete path for each file
path_files

In [ ]:
file_path = pathlib.Path(path_files[0])
video = imread(str(file_path)) 
img = np.expand_dims(video,axis=3)
image_optimization= np.concatenate((img,img),axis=3)

number_optimization_steps = 20

int_threshold_vector = np.round(np.linspace(0.1, 1.5, num=number_optimization_steps, endpoint=True,dtype=float),2)
cell_size_vector = np.linspace(60, 120, num=number_optimization_steps, endpoint=True,dtype=int)
model = models.Cellpose(gpu=True, model_type='nuclei') # model_type='cyto' or model_type='nuclei'
NUMBER_OF_CORES = np.min((2,multiprocessing.cpu_count()))
#NUMBER_OF_CORES = multiprocessing.cpu_count()

print(int_threshold_vector)
print(cell_size_vector)

In [ ]:
# Plotting the intensity distribution for a specific time point and an specific channel
plt.figure(figsize=(4,4))
plt.hist(image_optimization[0,:,:,0].flatten(), bins=80,color='orangered')
plt.xlabel('Intensity')
plt.ylabel('Frequency')
plt.ylim(0,1e5)
plt.title('Intensity Histogram')
plt.show()

In [ ]:
def cells_center_of_mass(list_masks_for_all_frames):
    # Creating dataframe with centers of mass
    list_df=[]
    number_frames= list_masks_for_all_frames.shape[0]
    for i in range(number_frames):
        number_cells_in_frame = list_masks_for_all_frames[i].max()
        for m in range (1, number_cells_in_frame+1):
            selected_mask = np.where(list_masks_for_all_frames[i]==m,1,0) # Selecting only one mask.  
            x,y = ndimage.measurements.center_of_mass(selected_mask)
            list_df.append( [i,int(x),int(y)] )
            del x, y, selected_mask
    return np.array(list_df)

In [ ]:
def convert_mask_labels (masks,tracking_dataframe):
    list_df=[]
    number_frames= masks.shape[0]
    list_new_masks= []
    for i in range(0,number_frames):
        # creating a copy of the original image
        temp_image = np.zeros_like(masks[0,:,:])
        trackpy_labels = tracking_dataframe.loc[tracking_dataframe['frame']==i ].particle.values
        
        # Iterate for each cell in 
        number_cells_in_frame = masks[i].max()
        for m in range (1, number_cells_in_frame+1):
            selected_mask = np.where(masks[i]==m,1,0) # Selecting only one mask.  
            for index, particle_label in enumerate (trackpy_labels):
                # Test if the center of mass is inside the selected mask 
                y, x = tracking_dataframe.loc[(tracking_dataframe['frame']==i ) & (tracking_dataframe['particle']==particle_label) , ['y', 'x']].values[0]
                if selected_mask[y,x]==1:
                    temp_image = np.where(selected_mask==1, particle_label,temp_image)
        list_new_masks.append(temp_image)
    return np.array(list_new_masks)

In [ ]:
def watershed_segmentation(img,min_distance=50):
    mask_image = img.copy()
    mask_image = (mask_image/mask_image.max()) * 255
    #mask_image[mask_image>=threshold] =255    
    plt.imshow(mask_image)
    new_mask = gaussian(mask_image, sigma=4) # applying the gaussian filter
    contours = measure.find_contours(new_mask, level=100, fully_connected='high') # Finding the contours in the image    
    # Connecting the last and first  elements in the array (contours) to get a fully connected shape
    contours_connected = np.vstack((contours))
    contours_connected = np.vstack((contours_connected[-1,:],contours_connected))
    # make a new mask from the contours array
    watershed_starting_mask = np.zeros(img.shape).astype(int)                    # Prealocating an array with zeros. Notice the datatype.
    rr, cc = polygon(contours_connected[:,0], contours_connected[:,1])   # Returns the coordinates inside the contour
    watershed_starting_mask[rr,cc] = 1   
    #Computes the Distance Transform distance in the image
    distance = - ndi.distance_transform_edt(watershed_starting_mask)        
    # Apply watershed
    distance = ndi.distance_transform_edt(watershed_starting_mask)                       # Computes the Distance Transform distance in the image
    coords = peak_local_max(distance, min_distance=min_distance, labels=watershed_starting_mask)   # Use the Distance transform image to find local maxima
    _,inds = np.unique(distance[coords[:,0],coords[:,1]],return_index=True)      # Make sure they are unique
    coords = coords[inds,:]                                                      # Selecting unique indexes
    mask = np.zeros(distance.shape, dtype=bool)                                  # Prealocating an array with zeros
    mask[tuple(coords.T)] = True                                                 # Make an image with 1's where local maxima are
    markers, _ = ndi.label(mask)                                                 # Unique values used as the desired labels
    # Using the watershed algorithm
    labels = watershed(-distance, markers, mask=watershed_starting_mask, watershed_line=True)  

    return labels

In [ ]:
# intensity_percentile_threshold = 95
# image_test = image_optimization.copy() # Making a copy of our img
# image_test[image_test>np.percentile(image_test,intensity_percentile_threshold)]=np.percentile(image_test,intensity_percentile_threshold)  # threshold image values larger than 1000 equal to 1000
# image_test = (image_test/image_test.max()) * 255
# labels = watershed_segmentation(image_test[10,:,:,1])
# plt.imshow(labels)

In [ ]:
def cellpose_segmentation(img,cell_size,model,int_threshold_vector):
    mask, _, _, _ = model.eval(img, diameter=cell_size, min_size=2000, channels=[0,0], net_avg=False, augment=False,flow_threshold=int_threshold_vector)
    return mask

In [ ]:
def cell_segmentation_optimized (image_to_segment,model, intensity_percentile_threshold=99,cell_size=100,minimal_trajectory_length=10, memory_tracking=0,masks_with_trackpy_labels= True,use_cellpose=True,min_distance=100): 
    # optimization code to detected cells using multiple intensity thresholds
    image_test = image_to_segment.copy() # Making a copy of our img
    image_test[image_test>np.percentile(image_test,96)]=np.percentile(image_test,96)  # threshold image values larger than 1000 equal to 1000
    
    if use_cellpose == True:
        list_masks = Parallel(n_jobs = NUMBER_OF_CORES)(delayed(cellpose_segmentation)(image_test[index_cellpose,:,:,:],cell_size,model,intensity_percentile_threshold) for index_cellpose in range(0,image_test.shape[0]))
        masks_for_all_frames = np.array(list_masks)
    else:
        list_masks = Parallel(n_jobs = NUMBER_OF_CORES)(delayed(watershed_segmentation)(image_test[index_cellpose,:,:,0]) for index_cellpose in range(0,image_test.shape[0]))
        masks_for_all_frames = np.array(list_masks)
    try:
        area_every_frame = np.count_nonzero(masks_for_all_frames, axis=0)
        array_center_mass = cells_center_of_mass(masks_for_all_frames)
        df = pd.DataFrame(array_center_mass, columns=['frame', 'y', 'x'])
        t = tp.link(df, memory=memory_tracking,search_range=80)
        t_filtered = tp.filter_stubs(t, minimal_trajectory_length)  # selecting trajectories that appear in at least 10 frames.
        num_detected_cells = t_filtered['particle'].nunique()-1 # removing the value for the empty cell
        
        # reordering the index for the particles to have consecutive numbers
        particle_index_trackpy_values = np.unique(t_filtered['particle'].values)
        for idx, particle_number in enumerate(particle_index_trackpy_values):
            t_filtered.loc[t_filtered['particle'] == particle_number, 'particle'] = - (idx+1)
        t_filtered['particle'] = t_filtered['particle'].abs()        
        trajectory_length =  np.array( [len(t_filtered.loc[t_filtered['particle'] == i, 'frame'].values) for i in range(1, t_filtered['particle'].nunique()+1  )] )
        metric =  np.mean(area_every_frame) * np.sum(trajectory_length) * num_detected_cells
        #metric =   np.sum(trajectory_length) * num_detected_cells
    except:
        num_detected_cells = 0
        t_filtered = pd.DataFrame([], columns=['frame', 'y', 'x'])
        metric = 0
        
    if (masks_with_trackpy_labels == True) and (num_detected_cells>0) :
        masks= convert_mask_labels (masks=masks_for_all_frames,tracking_dataframe=t_filtered)
        t_filtered.rename(columns={"particle": "Cell_ID"}, inplace=True)
    else:
        masks= masks_for_all_frames        
    return metric, masks, t_filtered, image_test,num_detected_cells

In [ ]:
# Optimization code
# detected_cells_matrix = np.zeros((int_threshold_vector.shape[0],cell_size_vector.shape[0] ) )
# for index_intensity, intensity_value in enumerate(tqdm(int_threshold_vector)):
#     for index_cell_size, cell_size in enumerate(cell_size_vector):
#         detected_cells_matrix[index_intensity,index_cell_size],_,_,_,_ = cell_segmentation_optimized (image_to_segment=image_optimization, intensity_percentile_threshold=intensity_value,cell_size=cell_size,model=model)

In [ ]:
# detected_cells_matrix = np.zeros((int_threshold_vector.shape[0],) )
# for index_intensity, intensity_value in enumerate(tqdm(int_threshold_vector)):
#     detected_cells_matrix[index_intensity],_,_,_ = cell_segmentation_optimized (image_to_segment=image_optimization, model=model, intensity_percentile_threshold=intensity_value,use_cellpose=False)

In [ ]:
# saving the numpy array
#np.save('detected_cells_matrix.npy', detected_cells_matrix)
#np.save('int_threshold_vector.npy', int_threshold_vector)
#np.save('cell_size_vector.npy', cell_size_vector)

In [ ]:
# loading the numpy array
detected_cells_matrix = np.load('detected_cells_matrix.npy') 
int_threshold_vector = np.load('int_threshold_vector.npy') 
cell_size_vector = np.load('cell_size_vector.npy') 

In [ ]:
# showing optimization plot
tick_values= np.linspace(0, number_optimization_steps,num=number_optimization_steps,endpoint=False)
df_optimization = pd.DataFrame(detected_cells_matrix ) # Converting the image into a pandas data frame
fig, ax = plt.subplots(1,1,figsize=(10,10))
sn.heatmap(df_optimization, annot=True,cmap="Spectral",ax=ax)
ax.set_xlabel('cell size')
ax.set_ylabel('intensity')
ax.set_yticks(tick_values,int_threshold_vector)  
ax.set_xticks(tick_values,cell_size_vector)
plt.show()

In [ ]:
# running for selected values
selected_intensity_index, selected_cell_size_index = unravel_index(detected_cells_matrix.argmax(), detected_cells_matrix.shape)
metric, masks_final, cell_movement_dataframe,image_optimization_selected,number_cells = cell_segmentation_optimized (image_to_segment=image_optimization, 
                                                                                                                intensity_percentile_threshold=int_threshold_vector[selected_intensity_index],
                                                                                                                cell_size=cell_size_vector[selected_cell_size_index], model=model ,
                                                                                                                masks_with_trackpy_labels= True)

In [ ]:
print(int_threshold_vector[selected_intensity_index], cell_size_vector[selected_cell_size_index], int(metric), number_cells)

In [ ]:
#4142 189 15
# metric, masks_final, cell_movement_dataframe,image_optimization_selected, number_cells = cell_segmentation_optimized (image_to_segment=image_optimization, 
#                                                                                                                 intensity_percentile_threshold=1.5,
#                                                                                                                 cell_size=116, 
#                                                                                                                 model=model ,
#                                                                                                                 masks_with_trackpy_labels= True)
# print(int_threshold_vector[selected_intensity_index], cell_size_vector[selected_cell_size_index], metric, number_cells)

In [ ]:
number_columns = 15
number_images = masks_final.shape[0]
number_rows = math.ceil(number_images/number_columns)
individual_figure_size = 2
# Loop to plot multiple cells in a grid
gs = gridspec.GridSpec(number_rows, number_columns)
gs.update(wspace = 0.1, hspace = 0.1) # set the spacing between axes.
fig = plt.figure(figsize = (individual_figure_size*number_columns, individual_figure_size*number_rows))
for index_image in range(0, number_images):
    ax = fig.add_subplot(gs[index_image])
    ax.imshow(masks_final[index_image], cmap = 'tab20',vmin=0,vmax=masks_final.max())
    ax.set_xticks([]); ax.set_yticks([])

In [ ]:
number_columns = 15
number_images = image_optimization_selected.shape[0]
number_rows = math.ceil(number_images/number_columns)
individual_figure_size = 2
# intensity

# Loop to plot multiple cells in a grid
gs = gridspec.GridSpec(number_rows, number_columns)
gs.update(wspace = 0.1, hspace = 0.1) # set the spacing between axes.
fig = plt.figure(figsize = (individual_figure_size*number_columns, individual_figure_size*number_rows))
for index_image in range(0, number_images):
    ax = fig.add_subplot(gs[index_image])
    ax.imshow(image_optimization_selected[index_image,:,:,0], cmap = 'Spectral',vmin=image_optimization_selected[:,:,:,0].min(),vmax=image_optimization_selected[:,:,:,0].max())
    ax.set_xticks([]); ax.set_yticks([])

### Plotting the spots in time

In [ ]:
def plotting_masks(index_frame=0):
    
    cells_ids = np.unique(cell_movement_dataframe['Cell_ID'].values)
    centers_mass = [cell_movement_dataframe.loc[(cell_movement_dataframe['Cell_ID']==selected_cell) & (cell_movement_dataframe['frame']==index_frame) ,['x','y','Cell_ID']].values for _,selected_cell in enumerate(cells_ids) ]
    array_center_mass = np.array ( [ele for ele in centers_mass if len(ele) >0])[:,0,:]
    
    fig, ax = plt.subplots(1,2,figsize=(10,5))
    # plotting real image
    ax[0].imshow(image_optimization[index_frame,:,:,0], cmap = 'Spectral_r')
    ax[0].set_title('Frame '+ str(index_frame))
    ax[0].set_xticks([]); ax[0].set_yticks([])
    
    # plotting masks
    ax[1].imshow(masks_final[index_frame], cmap = 'tab20',vmin=0,vmax=masks_final.max())
    ax[1].set_title('Frame '+ str(index_frame))
    ax[1].set_xticks([]); ax[1].set_yticks([])
    for i,cell in enumerate(array_center_mass[:,2]):
        x_val = array_center_mass[i,0]
        y_val = array_center_mass[i,1]
        ax[1].text(x=x_val-10, y=y_val-10, s=str(array_center_mass[i,2]), fontsize=12, color="Black" )
    plt.show()

In [ ]:
index_frame = widgets.IntSlider(value=0, min=0,max=masks_final.shape[0]-1,step=1, description='Frame:')
widgets.interact(plotting_masks, index_frame=index_frame);

In [ ]:
raise

## Plotting cell labels

## Detecting spots in the image

In [ ]:
image_test=image_optimization[:,:,:,0].copy()
image_test[image_test>np.percentile(image_test,99)]=np.percentile(image_test,99)  # threshold image values larger than 1000 equal to 1000
filtered_img = rsp.Utilities.log_filter(image=image_test, sigma=1)
print('filtered image shape ', filtered_img.shape)
print('filtered image range ', filtered_img.min(), filtered_img.max())

In [ ]:
def plotting_filtered_image(index_frame=0):
    fig, ax = plt.subplots(1,1,figsize=(10,10))
    # plotting real image
    ax.imshow(filtered_img[index_frame], cmap = 'Spectral_r')
    ax.set_title('Frame '+ str(index_frame))
    ax.set_xticks([])
    ax.set_yticks([])
    plt.show()

In [ ]:
index_frame = widgets.IntSlider(value=0, min=0,max=masks_final.shape[0]-1,step=1, description='Frame:')
widgets.interact(plotting_filtered_image, index_frame=index_frame);

In [ ]:
selected_percentile_to_detect_spots = 70
preprocess_image_for_tracking = True

In [ ]:
# This section generates an histogram with the intensity of the detected particles in the video.

particle_size = 7 # according to the documentation must be an odd number 3,5,7,9 etc.
minimal_intensity_for_selection = 0 # minimal intensity to detect a particle.

# "f" is a pandas data freame that contains the information about the detected spots
f = tp.locate(filtered_img[0,:,:], particle_size, minmass=minimal_intensity_for_selection,percentile=selected_percentile_to_detect_spots,preprocess=preprocess_image_for_tracking) 

plt.rcParams["figure.figsize"] = (5,5)
fig, ax = plt.subplots()
ax.hist(f['mass'], bins=100, color = "orangered", ec="orangered")
ax.set(xlabel='mass', ylabel='count')
ax.set_ylim([0,200])
plt.show()

In [ ]:
# This section generates an histogram with the intensity of the detected particles in the video.

particle_size = 7 # according to the documentation must be an odd number 3,5,7,9 etc.
minimal_intensity_for_selection = 3000 # minimal intensity to detect a particle.
# "f" is a pandas data frame that contains the information about the detected spots
fig, ax = plt.subplots(1,1,figsize=(10,10))

f = tp.locate(filtered_img[0,:,:], particle_size, minmass=minimal_intensity_for_selection,percentile=selected_percentile_to_detect_spots,preprocess=preprocess_image_for_tracking) 
tp.annotate(f,filtered_img[0,:,:],ax=ax); 

In [ ]:
# To start visualization move the time slider.
plt.rcParams["figure.figsize"] = (5,5) # if movie is too big, change size to (7,7)
def figure_viewer_tr(time,mass_text, particle_size):
    fig, ax = plt.subplots(1,1,figsize=(10,10))
    ch = 0  
    f = tp.locate(filtered_img[time],particle_size, minmass=mass_text, maxsize=7, percentile=selected_percentile_to_detect_spots,preprocess=preprocess_image_for_tracking) # "f" is a pandas data frame that contains the information about the detected spots
    tp.annotate(f,filtered_img[time],ax=ax) # tp.annotate is a trackpy function that displays the image with the detected spots  
values_size=[3,5,7,9] # Notice value must be an odd number.
interactive_plot_tr = interactive(figure_viewer_tr, mass_text = widgets.IntSlider(value=3000,min=1000,max=10000,description='min Intensity'), particle_size = widgets.IntSlider(min=3,max=11,step=2,value=7,description='Particle Size'), time = widgets.IntSlider(min=0,max=video.shape[0]-1,step=1,value=0,description='Time'))
controls = HBox(interactive_plot_tr.children[:-1], layout = Layout(flex_flow='row wrap'))
output = interactive_plot_tr.children[-1]
display(VBox([controls, output])) 

In [ ]:
# This section saves the parameters adjusted in the previous widget in two variables that will be use for the rest of the code.
selected_intensity = interactive_plot_tr.kwargs_widgets[1].value
selected_size = interactive_plot_tr.kwargs_widgets[2].value

In [ ]:
# "f" is a pandas data frame that contains the information about the detected spots.
# tp.batch is a trackpy function that detects spots for multiple frames in a video.
f = tp.batch(frames = filtered_img, diameter = selected_size, minmass = selected_intensity,percentile=selected_percentile_to_detect_spots,preprocess=preprocess_image_for_tracking)
#f = tp.batch(frames = filtered_img,kwargs = {'raw_image' : image_optimization[:,:,:,0]}, diameter = selected_size, minmass = selected_intensity,percentile=selected_percentile_to_detect_spots,preprocess=False)

In [ ]:
f

In [ ]:
def spots_in_mask(f, masks):
    f_test=f.copy()
    f_test['Cell_ID'] = 0
    frames_in_dataframe = np.unique(f_test['frame'].values)
    for _, selected_frame in enumerate(frames_in_dataframe):
        selected_frame_masks= masks[selected_frame]
        selected_dataframe = f_test.loc[f_test['frame']==selected_frame]
        coords_int = np.array([selected_dataframe.y, selected_dataframe.x], dtype= int).T # These are the points detected by trackpy
        values_at_coords = selected_frame_masks[tuple(coords_int.T)] #
        f_test.loc[f_test['frame']==selected_frame ,'Cell_ID'] = values_at_coords.T # Check if pts are on/in polygon mask  
        del selected_frame_masks, selected_dataframe, coords_int, values_at_coords
    return f_test 

In [ ]:
df_final = spots_in_mask(f, masks_final) 
df_final

In [ ]:
df_final = df_final[df_final['Cell_ID']>0] # removing spots outside masks
df_final

In [ ]:
maximum_search_range = 50

In [ ]:
cells_ids = np.unique(df_final['Cell_ID'].values)
list_df_cells =[]
for _, selected_cell in enumerate(cells_ids):
    df_selected_cell = df_final[df_final['Cell_ID']==selected_cell] # removing spots outside masks
    linked_dataframe = tp.link(df_selected_cell,search_range=maximum_search_range, memory=1)
    filtered_dataframe = tp.filter_stubs(linked_dataframe, 10)
    if len(filtered_dataframe)>0:
        list_df_cells.append(filtered_dataframe ) # selecting trajectories that appear in at least 10 frames.
    del linked_dataframe, df_selected_cell

In [ ]:
dfp = pd.DataFrame(data = None, columns= df_final.columns)
dfp = pd.concat(list_df_cells)

In [ ]:
dfp

In [ ]:
cells_ids = np.unique(df_final['Cell_ID'].values)

for _, selected_cell in enumerate(cells_ids):
    print('Cell_ID: ', selected_cell)
    array_particle_ids = np.unique(dfp.loc[dfp['Cell_ID'] == selected_cell, 'particle'].values)
    for _,particle_id in enumerate (array_particle_ids):
        frames_in_spot = dfp.loc[  (dfp['particle']==particle_id) &  (dfp['Cell_ID']==selected_cell) ].frame.values
        particle_intensity_in_spot =  dfp.loc[  (dfp['particle']==particle_id) &  (dfp['Cell_ID']==selected_cell) ].mass.values
        plt.plot(frames_in_spot,particle_intensity_in_spot)
    plt.show()

In [ ]:
# Defining the number of detected spots.
n_particles = dfp['particle'].nunique()

# Plotting
plt.figure(figsize=(20,5))

# plotting intensities in au
for id in range(0,n_particles):
    plt.plot(dfp.loc[dfp['particle']==dfp['particle'].unique()[id]].frame, dfp.loc[dfp['particle']==dfp['particle'].unique()[id]].mass )
plt.xlabel('Time (sec)')
plt.ylabel('Intensity a.u.')
plt.title('Intensity vs Time')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
ch = 0  
time =30
tp.annotate(dfp,filtered_img[time],ax=ax)